# Dealing with errors after a run

In this example, we run the model on a list of three glaciers:
two of them will end with errors: one because it already failed at
preprocessing (i.e. prior to this run), and one during the run. We show how to analyze theses erros and solve (some) of them, as described in the OGGM documentation under [troubleshooting](https://docs.oggm.org/en/stable/faq.html?highlight=border#troubleshooting).

## Run with `cfg.PARAMS['continue_on_error'] = True`

In [ ]:
# Locals
import oggm.cfg as cfg
from oggm import utils, workflow, tasks

# Libs
import os
import xarray as xr
import pandas as pd

# Initialize OGGM and set up the default run parameters
cfg.initialize(logging_level='WARNING')

# Here we override some of the default parameters
# How many grid points around the glacier?
# We make it small because we want the model to error because
# of flowing out of the domain
cfg.PARAMS['border'] = 80

# This is useful since we have three glaciers
cfg.PARAMS['use_multiprocessing'] = True

# This is the important bit!
# We tell OGGM to continue despite of errors
cfg.PARAMS['continue_on_error'] = True

# Local working directory (where OGGM will write its output)
WORKING_DIR = utils.gettempdir('OGGM_Errors')
utils.mkdir(WORKING_DIR, reset=True)
cfg.PATHS['working_dir'] = WORKING_DIR

rgi_ids = ['RGI60-11.00897', 'RGI60-11.01450', 'RGI60-11.03295']

# Go - get the pre-processed glacier directories
gdirs = workflow.init_glacier_directories(rgi_ids, from_prepro_level=4)

# We can step directly to the experiment!
# Random climate representative for the recent climate (1985-2015)
# with a negative bias added to the random temperature series
workflow.execute_entity_task(tasks.run_random_climate, gdirs,
                             nyears=150, seed=0,
                             temperature_bias=-1)

## Error diagnostics

In [ ]:
# Write the compiled output
utils.compile_glacier_statistics(gdirs);  # saved as glacier_statistics.csv in the WORKING_DIR folder
utils.compile_run_output(gdirs);  # saved as run_output.nc in the WORKING_DIR folder

In [ ]:
# Read it
with xr.open_dataset(os.path.join(WORKING_DIR, 'run_output.nc')) as ds:
    ds = ds.load()
df_stats = pd.read_csv(os.path.join(WORKING_DIR, 'glacier_statistics.csv'), index_col=0)

In [ ]:
# all possible statistics about the glaciers
df_stats

- in the column *error_task*, we can see whether an error occurred, and if yes during which task
- *error_msg* describes the actual error message 

In [ ]:
df_stats[['error_task', 'error_msg']]

We can also check which glacier failed at which task by using [compile_task_log]('https://docs.oggm.org/en/stable/generated/oggm.utils.compile_task_log.html#oggm.utils.compile_task_log').

In [ ]:
# also saved as task_log.csv in the WORKING_DIR folder - "append=False" replaces the existing one
utils.compile_task_log(gdirs, task_names=['glacier_masks', 'compute_centerlines', 'flowline_model_run'], append=False)

## Error solving

### RuntimeError: `Glacier exceeds domain boundaries, at year: 98.08333333333333`

To remove this error just increase the domain boundary **before** running `init_glacier_directories` ! Attention, this means that more data has to be downloaded and the run takes more time. The available options for `cfg.PARAMS['border']` are **10, 40, 80 or 160** at the moment; the unit is number of grid points outside the glacier boundaries. More about that in the OGGM documentation under [preprocessed files](https://docs.oggm.org/en/stable/input-data.html#pre-processed-directories).

In [ ]:
# reset to recompute statistics
utils.mkdir(WORKING_DIR, reset=True)

# increase the amount of gridpoints outside the glacier
cfg.PARAMS['border'] = 160
gdirs = workflow.init_glacier_directories(rgi_ids, from_prepro_level=4)
workflow.execute_entity_task(tasks.run_random_climate, gdirs,
                             nyears=150, seed=0,
                             temperature_bias=-1);

# recompute the output
# we can also get the run output directly from the methods
df_stats = utils.compile_glacier_statistics(gdirs)
ds = utils.compile_run_output(gdirs)

In [ ]:
# check again
df_stats[['error_task', 'error_msg']]

Now `RGI60-11.00897` runs without errors!

### Error: `Need a valid model_flowlines file.`

This error message in the log is misleading: it does not really describe the source of the error, which happened earlier in the processing chain. Therefore we can look instead into the glacier_statistics via [compile_glacier_statistics](https://docs.oggm.org/en/stable/generated/oggm.utils.compile_glacier_statistics.html) or into the log output via [compile_task_log](https://docs.oggm.org/en/stable/generated/oggm.utils.compile_task_log.html#oggm.utils.compile_task_log):

In [ ]:
print('error_task: {}, error_msg: {}'.format(df_stats.loc['RGI60-11.03295']['error_task'],
                                             df_stats.loc['RGI60-11.03295']['error_msg']))

Now we have a better understanding of the error: 
- OGGM can not work with this geometry of this glacier and could therefore not make a gridded mask of the glacier outlines. 
- there is no way to prevent this except you find a better way to pre-process the geometry of this glacier
- these glaciers have to be ignored! Less than 0.5% of glacier area globally have errors during the geometry processing or failures in computing certain topographical properties by e.g. invalid DEM, see [Sect. 4.2 Invalid Glaciers of the OGGM paper (Maussion et al., 2019)](https://gmd.copernicus.org/articles/12/909/2019/#section4) and [this tutorial](preprocessing_errors.ipynb) for more up-to-date numbers

## Ignoring those glaciers with errors that we can't solve

In the run_output, you can for example just use `*.dropna` to remove these. For other applications (e.g. quantitative mass change evaluation), more will be needed (not available yet in the OGGM codebase):

In [ ]:
ds.dropna(dim='rgi_id')  # here we can e.g. find the volume evolution

## What's next?

- read about [preprocessing errors](preprocessing_errors.ipynb)
- return to the [OGGM documentation](https://docs.oggm.org)
- back to the [table of contents](welcome.ipynb)